In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [3]:
!pip install findspark

In [4]:
import findspark

In [5]:
findspark.init('spark-3.5.0-bin-hadoop3')

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode

# создание SparkSession
spark = SparkSession.builder.appName("ProductCategory").getOrCreate()

# пример DataFrame с данными
data = [("Продукт1", ["Категория1", "Категория2"]),
        ("Продукт2", ["Категория2", "Категория3"]),
        ("Продукт3", []),
        ("Продукт4", ["Категория1"]),
        ("Продукт5", ["Категория4"])]

schema = ["Название продукта", "Категория"]

df = spark.createDataFrame(data, schema)

# разделение списока категорий на отдельные строки
df_exploded = df.select(col("Название продукта"), explode(col("Категория")).alias("Категория"))

# приведение к одномутипу данных
df_exploded = df_exploded.withColumn("Категория", df_exploded["Категория"].cast("string"))

# создание DataFrame с продуктами без категорий
df_with_empty_categories = df.filter((col("Категория").cast("string") == "[]"))

# приведение к одномутипу данных
df_with_empty_categories = df_with_empty_categories.withColumn("Категория", df_with_empty_categories["Категория"].cast("string"))

# объеденение двух DataFrames
result_df = df_exploded.union(df_with_empty_categories)

result_df.show()


+-----------------+----------+
|Название продукта| Категория|
+-----------------+----------+
|         Продукт1|Категория1|
|         Продукт1|Категория2|
|         Продукт2|Категория2|
|         Продукт2|Категория3|
|         Продукт4|Категория1|
|         Продукт5|Категория4|
|         Продукт3|        []|
+-----------------+----------+

